In [1]:
import pandas as pd
import altair as alt

def plot_yearly_passenger_trends(start_year):
    all_data = []

    # Load each year's file
    for year in range(start_year, 2026):
        try:
            df = pd.read_csv(f"clean_data/flights_{year}_clean.csv")

            # Aggregate to monthly totals
            monthly = df.groupby('MONTH')['PASSENGERS'].sum().reset_index()
            monthly['YEAR'] = year
            monthly['date'] = pd.to_datetime(
                monthly['YEAR'].astype(str) + "-" + monthly['MONTH'].astype(str) + "-01"
            )

            all_data.append(monthly)
        except FileNotFoundError:
            print(f"Missing file for {year} — skipping.")
            continue

    # Combine all years
    if not all_data:
        raise ValueError("No data loaded.")
    data = pd.concat(all_data, ignore_index=True)

    # --------------------------
    # Crisis + recovery intervals
    # --------------------------
    highlight_periods = pd.DataFrame({
        'start': [
            '2001-01-01',  # 9/11
            '2008-01-01',  # financial crisis
            '2019-10-01'   # covid
        ],
        'end': [
            '2002-01-31',
            '2009-12-31',
            '2021-01-01'
        ],
        'event': [
            '9/11 Aftermath',
            '2008 Financial Crisis',
            'COVID-19 Impact'
        ]
    })

    highlight_periods['start'] = pd.to_datetime(highlight_periods['start'])
    highlight_periods['end']   = pd.to_datetime(highlight_periods['end'])

    # Shaded rectangles
    rects = alt.Chart(highlight_periods).mark_rect(
        opacity=0.25,
        color='red'
    ).encode(
        x='start:T',
        x2='end:T'
    )

    # Passenger line
    line = alt.Chart(data).mark_line(point=True).encode(
        x=alt.X('date:T', title='Date'),
        y=alt.Y('PASSENGERS:Q', title='Total Monthly Passengers'),
        color=alt.value('#1f77b4'),
        tooltip=[
            alt.Tooltip('date:T', title='Month'),
            alt.Tooltip('PASSENGERS:Q', title='Passengers', format=','),
        ]
    )

    # Event labels at the top of shaded regions
    labels = alt.Chart(highlight_periods).mark_text(
        align='left',
        baseline='bottom',
        dx=3,
        dy=3,
        fontSize=12,
        fontWeight='bold'
    ).encode(
        x='start:T',
        y=alt.value(10),
        text='event'
    )

    chart = (rects + line + labels).properties(
        width=900,
        height=400,
        title="Monthly Passenger Trends Over Time"
    )

    return chart



In [2]:
plot_yearly_passenger_trends(1999)

alt.LayerChart(...)